<a href="https://colab.research.google.com/github/MrMazagngy/NFT-Value-Estimation-/blob/main/NFT_Value_Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from urllib.request import urlopen
from PIL import Image
import numpy as np
import requests
import cv2
import pandas as pd
import glob
import math
#Model Architcture
import tensorflow as tf
import keras
from keras.layers import Input, Flatten , Dense ,LSTM ,Dropout
from keras.models import Model

In [ ]:
'''
Cryptopunk Collection Has 10,000 NFTs , so we loop through all of them
And Get Only Those With Previous Sales
'''
for i in range (1,10000):
  
  url = "https://api.opensea.io/api/v1/asset/0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb/"+ str(i)
  response = requests.request("GET", url).json()

  if response['last_sale'] != None :

    print (i)
    price = int(response['last_sale']['total_price']) / 1000000000000000000   #Prices Have 18 Decimals 
    with open('/content/drive/My Drive/NFT Value Estimation/price.txt', 'a+') as f: #create a text file if not existed with name (price.txt)
      f.write(str(price)+'\n')

    img = Image.open(urlopen(response['image_url']))
    img = np.array(img)
    img_name = '/content/drive/My Drive/NFT Value Estimation/pic_'+str(i) # save images with this format {pic_number}
    cv2.imwrite(img_name +'.png' , img)


In [3]:
#Prices for all extracted NFTs
Y = np.loadtxt('/content/drive/My Drive/NFT Value Estimation/price.txt')


In [4]:
#Retrieve all images from its folder and store them in a list
X = []

images = glob.glob('/content/drive/My Drive/NFT Value Estimation/*.png')
for  image in images :
  X.append(image)

In [5]:
#Splitting Data into Tain / Validation / Test Sets
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size = 0.3, random_state = 1)

In [6]:
'''
Custom Data Generator class to preprocess images to ResNet50 shape of (224,224) and normalize its values

'''
class CustomDataGen(tf.keras.utils.Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

        return np.array([
            cv2.resize(cv2.imread(file_name), (224, 224))
               for file_name in batch_x]), np.array(batch_y)

In [7]:
Batch_size = 32

train = CustomDataGen(X_train,y_train,Batch_size)
validation = CustomDataGen(X_val,y_val,Batch_size)
test = CustomDataGen(X_test,y_test,Batch_size)

In [ ]:
#Model Architcture 
input = Input(shape=(224, 224, 3))

base = tf.keras.applications.ResNet50(include_top=False,weights="imagenet" )
base.trainable=False
x = Flatten()(base(input))
x = tf.expand_dims(x, -1)
x = Dropout(0.3)(x)
x = LSTM(50)(x)
x = Dense(10)(x)
x = Dense(1)(x)

model = Model(inputs=input, outputs=x)
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.summary()

In [ ]:
Epochs = 10

# define the checkpoint
filepath = "'/content/drive/My Drive/NFT Value Estimation/models.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(train,epochs=Epochs, validation_data = validation,callbacks=callbacks_list)